## Downsampling Analysis

### counts downsampleing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import decoupler as dc
import scanpy as sc
import os
import math
import csv

In [ ]:
PATH='/filtered_feature_bc_matrix/' 
ap={}
sample=['DUR_JE','SS_JE','JH_JE']

for i in sample:
    ap[f'{i}']=sc.read_10x_mtx(PATH+f'{i}')
    ap[f'{i}'].var_names_make_unique
    scrublets=pd.read_csv(PATH+f'{i}'+'/doublet.txt',index_col='barcode')
    ap[f'{i}'].obs['doublet_scores']=scrublets['doublet_scores']
    ap[f'{i}'].obs['predicted_doublets']=scrublets['predicted_doublets']
    x=['{}',f'{i}']
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.map('_'.join(x).format)
    
    sc.pp.filter_cells(ap[f'{i}'], min_genes=200)
    sc.pp.filter_genes(ap[f'{i}'], min_cells=3)
    ap[f'{i}'].var['mt'] = ap[f'{i}'].var_names.str.startswith('MT') 
    sc.pp.calculate_qc_metrics(ap[f'{i}'], qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    ap[f'{i}'] = ap[f'{i}'][ap[f'{i}'].obs['predicted_doublets']==False, :].copy()
    ap[f'{i}'] = ap[f'{i}'][ap[f'{i}'].obs.n_genes_by_counts < 7500, :]
    ap[f'{i}'] = ap[f'{i}'][ap[f'{i}'].obs.n_genes_by_counts > 200, :]
    ap[f'{i}'] = ap[f'{i}'][ap[f'{i}'].obs.pct_counts_mt < 0.5, :]

In [ ]:
ap['JH_JE'].obs['total_counts'].describe()

In [ ]:
ap['DUR_JE'].obs['total_counts'].describe()

In [ ]:
ap['SS_JE'].obs['total_counts'].describe()

In [ ]:
adata_jh=ap['JH_JE'].copy()
adata_du=ap['DUR_JE'].copy()
adata_awb=ap['SS_JE'].copy()

In [ ]:
sc.pp.downsample_counts(adata_jh, counts_per_cell=6300, random_state=2, replace=False, copy=False)
sc.pp.downsample_counts(adata_du, counts_per_cell=6300, random_state=2, replace=False, copy=False)
sc.pp.downsample_counts(adata_awb, counts_per_cell=6300, random_state=2, replace=False, copy=False)

In [ ]:
sc.pp.downsample_counts(adata_jh, counts_per_cell=8172, random_state=2, replace=False, copy=False)
sc.pp.downsample_counts(adata_du, counts_per_cell=8172, random_state=2, replace=False, copy=False)
sc.pp.downsample_counts(adata_awb, counts_per_cell=8172, random_state=2, replace=False, copy=False)

In [ ]:
sc.pp.calculate_qc_metrics(adata_jh, percent_top=None, log1p=False, inplace=True)
sc.pp.calculate_qc_metrics(adata_du, percent_top=None, log1p=False, inplace=True)
sc.pp.calculate_qc_metrics(adata_awb, percent_top=None, log1p=False, inplace=True)

In [ ]:
concat = sc.read_h5ad('adata_jh_downsampling_mid_6300.h5ad')
adata = sc.read_h5ad('adata_CellLineage.h5ad')
common_cells = set(concat.obs_names) & set(adata.obs_names)
common_genes = set(concat.var_names) & set(adata.var_names)
concat = concat[concat.obs_names.isin(common_cells), :]
adata = adata[adata.obs_names.isin(common_cells), :]
concat = concat[: ,concat.var_names.isin(common_genes)]
adata = adata[:, adata.var_names.isin(common_genes)]
adata.layers['counts']=concat.X

In [ ]:
pdata = dc.get_pseudobulk(
    adata,
    sample_col='BREED',
    groups_col=None,
    layer='counts',
    mode='sum',
    min_cells=10,
    min_counts=1000
)

In [ ]:
pdata.layers['counts'] = pdata.X.copy()

In [ ]:
list = pdata.T.to_df()
list.to_csv("pseudobulk_raw_counts_jh_downsampling_mid_6300_"+i+".csv", index=True)

### cell number downsampleing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import decoupler as dc
import scanpy as sc
import os
import math
import csv

In [ ]:
PATH='/filtered_feature_bc_matrix/' 
ap={}
sample=['DUR_JE','SS_JE','JH_JE']

for i in sample:
    ap[f'{i}']=sc.read_10x_mtx(PATH+f'{i}')
    ap[f'{i}'].var_names_make_unique
    scrublets=pd.read_csv(PATH+f'{i}'+'/doublet.txt',index_col='barcode')
    ap[f'{i}'].obs['doublet_scores']=scrublets['doublet_scores']
    ap[f'{i}'].obs['predicted_doublets']=scrublets['predicted_doublets']
    x=['{}',f'{i}']
    ap[f'{i}'].obs.index=ap[f'{i}'].obs.index.map('_'.join(x).format)
    
    sc.pp.filter_cells(ap[f'{i}'], min_genes=200)
    sc.pp.filter_genes(ap[f'{i}'], min_cells=3)
    ap[f'{i}'].var['mt'] = ap[f'{i}'].var_names.str.startswith('MT') 
    sc.pp.calculate_qc_metrics(ap[f'{i}'], qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    ap[f'{i}'] = ap[f'{i}'][ap[f'{i}'].obs['predicted_doublets']==False, :].copy()
    ap[f'{i}'] = ap[f'{i}'][ap[f'{i}'].obs.n_genes_by_counts < 7500, :]
    ap[f'{i}'] = ap[f'{i}'][ap[f'{i}'].obs.n_genes_by_counts > 200, :]
    ap[f'{i}'] = ap[f'{i}'][ap[f'{i}'].obs.pct_counts_mt < 0.5, :]

In [8]:
ap['JH_JE'].write('adata_jh_rowdata.h5ad')
ap['DUR_JE'].write('adata_du_rowdata.h5ad')
ap['SS_JE'].write('adata_awb_rowdata.h5ad')

... storing 'feature_types' as categorical
... storing 'feature_types' as categorical
... storing 'feature_types' as categorical


In [361]:
adata_jh = ad.read('adata_jh_rowdata.h5ad')
adata_du = ad.read('adata_du_rowdata.h5ad')
adata_awb = ad.read('adata_awb_rowdata.h5ad')

In [ ]:
sc.pp.subsample(adata_jh, n_obs=2000, random_state=1)
sc.pp.subsample(adata_du, n_obs=2000, random_state=1)
sc.pp.subsample(adata_awb, n_obs=2000, random_state=1)

In [363]:
adata_jh.write('adata_jh_subsample_1.h5ad')
adata_du.write('adata_du_subsample_1.h5ad')
adata_awb.write('adata_awb_subsample_1.h5ad')

In [ ]:
adata_jh = ad.read('adata_jh_subsample_1.h5ad')
adata_du = ad.read('adata_du_subsample_1.h5ad')
adata_awb = ad.read('adata_awb_subsample_1.h5ad')

adata_combined = ad.concat([adata_jh, adata_du, adata_awb], label='sample', keys=['JH', 'DUR', 'SS'])

adata_combined.write('adata_combined.h5ad')


In [ ]:
concat = sc.read_h5ad('adata_combined.h5ad')
adata = sc.read_h5ad('adata_CellLineage.h5ad')
common_cells = set(concat.obs_names) & set(adata.obs_names) 
common_genes = set(concat.var_names) & set(adata.var_names)
concat = concat[concat.obs_names.isin(common_cells), :]
adata = adata[adata.obs_names.isin(common_cells), :]
concat = concat[: ,concat.var_names.isin(common_genes)]
adata = adata[:, adata.var_names.isin(common_genes)]

In [ ]:
adata_plasma=adata[adata.obs['CellLineage'].isin(['Plasma'])]
jh_plasma=adata_plasma[adata_plasma.obs['BREED'].isin(['JH'])]
common_cells = set(concat.obs_names) & set(jh_plasma.obs_names) 
common_genes = set(concat.var_names) & set(jh_plasma.var_names)
concat = concat[concat.obs_names.isin(common_cells), :]
jh_plasma = jh_plasma[jh_plasma.obs_names.isin(common_cells), :]
concat = concat[: ,concat.var_names.isin(common_genes)]
jh_plasma = jh_plasma[:, jh_plasma.var_names.isin(common_genes)]

In [369]:
adata = sc.read_h5ad('adata_CellLineage.h5ad')

In [370]:
adata = adata[adata.obs['CellLineage'].isin(['Plasma'])]
adata = adata[adata.obs['BREED'].isin(['JH'])]

In [ ]:
adata_jh = ad.read('adata_jh_rowdata.h5ad')
adata_du = ad.read('adata_du_rowdata.h5ad')
adata_awb = ad.read('adata_awb_rowdata.h5ad')

adata_combined = ad.concat([adata_jh, adata_du, adata_awb], label='sample', keys=['JH', 'DU', 'AWB'])

In [ ]:
common_cells = set(adata_combined.obs_names) & set(adata.obs_names)
common_genes = set(adata_combined.var_names) & set(adata.var_names)
adata_combined = adata_combined[adata_combined.obs_names.isin(common_cells), :]
adata = adata[adata.obs_names.isin(common_cells), :]
adata_combined = adata_combined[: ,adata_combined.var_names.isin(common_genes)]
adata = adata[:, adata.var_names.isin(common_genes)]

In [373]:
adata.layers['counts'] = adata_combined.X
pdata = dc.get_pseudobulk(
    adata,
    sample_col='BREED',
    groups_col=None,
    layer='counts',
    mode='sum',
    min_cells=10,
    min_counts=1000
)

In [374]:
list = pdata.T.to_df()
list.to_csv("pseudobulk_raw_counts_jh_downsampling_1.csv", index=True)

In [375]:
concat.layers['counts'] = concat.X

In [376]:
pdata = dc.get_pseudobulk(
    concat,
    sample_col='sample',
    groups_col=None,
    layer='counts',
    mode='sum',
    min_cells=10,
    min_counts=1000
)

In [377]:
list = pdata.T.to_df()
list.to_csv("pseudobulk_counts_jh_downsampling_1.csv", index=True)

### correlation coefficient

In [ ]:
import pandas as pd
from scipy.stats import pearsonr

jh_raw_counts = pd.read_csv('pseudobulk_counts_jh_downsampling_1.csv', index_col=0)
jh_counts = pd.read_csv('pseudobulk_raw_counts_jh_downsampling_1.csv', index_col=0)

print(jh_raw_counts.head())
print(jh_counts.head())

common_genes = jh_raw_counts.index.intersection(jh_counts.index)

jh_raw_counts_subset = jh_raw_counts.loc[common_genes]
jh_counts_subset = jh_counts.loc[common_genes]

correlation, p_value = pearsonr(jh_raw_counts_subset['JH'], jh_counts_subset['JH'])

print(f'Pearson Correlation: {correlation}')
print(f'P-value: {p_value}')


          JH
A1CF    38.0
A4GALT  14.0
A4GNT    3.0
AAAS    51.0
AACS    47.0
          JH
A1CF    42.0
A2ML1    1.0
A4GALT  22.0
A4GNT    3.0
AAAS    70.0
Pearson Correlation: 0.9999959456488959
P-value: 0.0
